# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

## Question 1 : What is the sum of the outputs of the generator for limit = 5?

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

def sum_square_root_generator(limit) :
    sum = 0
    generator = square_root_generator(limit)

    for sqrt_value in generator:
        sum += sqrt_value
        yield sum

limit = 5

generator = sum_square_root_generator(limit)

for sum in generator :
    print(sum)

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


## Question 2 : What is the 13th number yielded by the generator?

In [2]:
limit = 13

generator = square_root_generator(limit)

for i, sqrt_num in enumerate(generator) :
    print(i+1,sqrt_num)


1 1.0
2 1.4142135623730951
3 1.7320508075688772
4 2.0
5 2.23606797749979
6 2.449489742783178
7 2.6457513110645907
8 2.8284271247461903
9 3.0
10 3.1622776601683795
11 3.3166247903554
12 3.4641016151377544
13 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




## Question 3 : Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [30]:
%%capture
%pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [50]:
import dlt

#Creatin table people from generator people_1
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

In [51]:
#Appending data from generator people_2 to existing table

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

In [52]:
import duckdb

con = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

In [54]:
con.sql("select sum(Age) from people.people")

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


## Question 4 : Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [69]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = generators_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="replace",
                    primary_key = "id")

In [70]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = generators_pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key = "id")

In [73]:
con.sql('select count(*) from people.people_merge')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            8 │
└──────────────┘

In [74]:
con.sql('select Age from people.people_merge where id = 3')

┌───────┐
│  age  │
│ int64 │
├───────┤
│    33 │
└───────┘

In [72]:
con.sql('select sum(Age) from people.people_merge')

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘